In [9]:
# -*- coding: utf-8 -*-
"""
LightGBM + CV + SHAP  完整模板
行号一一对应，X 仅缺值已补，y 为 WOMKP 0/1
"""
import pandas as pd
import pathlib
import numpy as np
import lightgbm as lgb
import shap
import matplotlib.pyplot as plt

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

# ========= 1. 路径设置 =========
data_dir = pathlib.Path(r"C:\Users\DXW\Desktop\半月板部分切除术\OAI test\test 3\Xrayandclinical")
x_file   = data_dir / 'pred_processed.xlsx'   # 已补缺失
y_file   = data_dir / 'outcome.xlsx'  # 含 WOMKP 0/1
oof_file = data_dir / 'LightGBM_OOF_prob.xlsx'
shap_summary_path = data_dir / 'SHAP_summary.png'
shap_bar_path     = data_dir / 'SHAP_bar.png'
shap_force_path   = data_dir / 'SHAP_force.html'

# ========= 2. 读数据 =========
X_df = pd.read_excel(x_file, sheet_name="Sheet1").drop(columns=['ID','MONTHS','side',], errors='ignore')
y_df = pd.read_excel(y_file)[['Final']].values.ravel()

# ========= 3. 把列分成 3 类 =========
bin_like = [c for c in X_df.columns if X_df[c].dropna().isin([0,1]).all()]
continuous_cols = [
    c for c in X_df.select_dtypes(include=['int64','float64']).columns
    if c not in bin_like
]
ordinal_cols = []          # 例：['KL_grade', 'pain_scale']
no_scale_cols = bin_like + ordinal_cols

scale_pipe   = Pipeline([('scaler', StandardScaler())])
passthrough_pipe = Pipeline([('identity', 'passthrough')])

preprocessor = ColumnTransformer(
    transformers=[
        ('scale', scale_pipe, continuous_cols),
        ('passthrough', passthrough_pipe, no_scale_cols)
    ],
    remainder='drop'
)

# ========= 5. LightGBM 参数 =========
lgb_params = {
    'objective': 'binary',
    'metric': 'auc',
    'max_depth': 5,
    'learning_rate': 0.05,
    'subsample': 1,
    'colsample_bytree': 1,
    'random_state': 42,
    'n_jobs': -1,
    'verbose': -1
}

# ========= 6. 交叉验证 =========
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
auc_scores = []
oof_prob = np.zeros(len(X_df))
trained_models, val_indices = [], []

for fold, (tr_idx, val_idx) in enumerate(cv.split(X_df, y_df)):
    print(f'\n===== Fold {fold+1}/5 =====')
    X_train_df, X_val_df = X_df.iloc[tr_idx], X_df.iloc[val_idx]
    y_train, y_val = y_df[tr_idx], y_df[val_idx]

    # 预处理
    X_train = preprocessor.fit_transform(X_train_df)
    X_val   = preprocessor.transform(X_val_df)

    # 训练
    dtrain = lgb.Dataset(X_train, label=y_train)
    dval   = lgb.Dataset(X_val, label=y_val)
    model = lgb.train(
        params=lgb_params,
        train_set=dtrain,
        num_boost_round=200,
        valid_sets=[dval],
        callbacks=[lgb.early_stopping(50), lgb.log_evaluation(0)]
    )

    # 预测
    val_prob = model.predict(X_val)
    oof_prob[val_idx] = val_prob
    auc = roc_auc_score(y_val, val_prob)
    auc_scores.append(auc)
    print(f'Fold {fold+1} AUC = {auc:.4f}')

    # 存模型和验证索引（SHAP 用）
    trained_models.append(model)
    val_indices.append(val_idx)

# ========= 7. CV 结果 =========
print('\n===== CV 结果 =====')
print('5 折 AUC :', np.round(auc_scores, 4))


# ========= 8. 保存 OOF 概率 =========
pd.DataFrame({'OOF_prob': oof_prob, 'label': y_df}).to_excel(oof_file, index=False)
print('OOF 概率已保存：', oof_file)

# ========= 9. SHAP 解释（逐折算 → 折间平均 ± 标准差） =========
print('\n===== SHAP 解释（5-fold mean ± std） =====')

fold_imp = []   # 每折的 mean(|SHAP|)
for fold, (model, v_idx) in enumerate(zip(trained_models, val_indices)):
    X_val = preprocessor.transform(X_df.iloc[v_idx])
    explainer = shap.TreeExplainer(model)
    shap_val = explainer.shap_values(X_val)        # (n_val_samples, n_features)
    fold_imp.append(np.abs(shap_val).mean(axis=0)) # 该折平均绝对值

# 折间统计
mean_imp = np.mean(fold_imp, axis=0)   # (n_features,)
std_imp  = np.std(fold_imp, axis=0)
feat_names = preprocessor.get_feature_names_out()
shap_df = (pd.DataFrame({'mean': mean_imp, 'std': std_imp}, index=feat_names)
             .sort_values('mean', ascending=False))

# 画 top30 带误差条
top30 = shap_df.head(30)
plt.ioff()
plt.figure(figsize=(6, 8))
y_pos = np.arange(len(top30))
plt.barh(y_pos, top30['mean'], xerr=top30['std'], capsize=3,
         color='steelblue', alpha=0.8)
plt.yticks(y_pos, top30.index)
plt.xlabel('mean(|SHAP|) across 5 folds')
plt.title('SHAP importance — 5-fold mean ± std')
plt.tight_layout()
plt.savefig(shap_bar_path, dpi=300)
plt.close()

# beeswarm 图：把 5 折验证集拼起来后用第 1 折模型画一次即可
X_val_concat = np.vstack([preprocessor.transform(X_df.iloc[v_idx])
                          for v_idx in val_indices])
explainer0 = shap.TreeExplainer(trained_models[0])
shap_values_concat = explainer0.shap_values(X_val_concat)
shap.summary_plot(shap_values_concat, X_val_concat,
                  feature_names=feat_names, show=False)
plt.savefig(shap_summary_path, bbox_inches='tight')
plt.close()

# 个体 force plot（合并后第 0 条样本）
html_force = shap.force_plot(explainer0.expected_value,
                             shap_values_concat[0, :],
                             X_val_concat[0, :],
                             feature_names=feat_names)
shap.save_html(str(shap_force_path), html_force)

print('SHAP 结果已生成（5 折平均 ± 标准差）：')
print(' ', shap_summary_path)
print(' ', shap_bar_path)
print(' ', shap_force_path)
print('Mean AUC :', np.mean(auc_scores))
print('Std  AUC :', np.std(auc_scores))



===== Fold 1/5 =====
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[5]	valid_0's auc: 0.913462
Fold 1 AUC = 0.9135

===== Fold 2/5 =====
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[117]	valid_0's auc: 0.889423
Fold 2 AUC = 0.8894

===== Fold 3/5 =====
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[23]	valid_0's auc: 0.758242
Fold 3 AUC = 0.7582

===== Fold 4/5 =====
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[5]	valid_0's auc: 0.868132
Fold 4 AUC = 0.8681

===== Fold 5/5 =====
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[30]	valid_0's auc: 0.923077
Fold 5 AUC = 0.9231

===== CV 结果 =====
5 折 AUC : [0.9135 0.8894 0.7582 0.8681 0.9231]
OOF 概率已保存： C:\Users\DXW\Desktop\半月板部分切除术\OAI test\test 3\Xrayandclinical\LightGBM_OOF_prob.xlsx

===

c:\Users\DXW\miniconda3\envs\dicom\lib\site-packages\shap\explainers\_tree.py:586: UserWarning: LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray
  warnings.warn(
c:\Users\DXW\miniconda3\envs\dicom\lib\site-packages\shap\explainers\_tree.py:586: UserWarning: LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray
  warnings.warn(
c:\Users\DXW\miniconda3\envs\dicom\lib\site-packages\shap\explainers\_tree.py:586: UserWarning: LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray
  warnings.warn(
c:\Users\DXW\miniconda3\envs\dicom\lib\site-packages\shap\explainers\_tree.py:586: UserWarning: LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray
  warnings.warn(
c:\Users\DXW\miniconda3\envs\dicom\lib\site-packages\shap\explainers\_tree.py:586: UserWarning: LightGBM binary classifier with TreeExplainer shap values ou

SHAP 结果已生成（5 折平均 ± 标准差）：
  C:\Users\DXW\Desktop\半月板部分切除术\OAI test\test 3\Xrayandclinical\SHAP_summary.png
  C:\Users\DXW\Desktop\半月板部分切除术\OAI test\test 3\Xrayandclinical\SHAP_bar.png
  C:\Users\DXW\Desktop\半月板部分切除术\OAI test\test 3\Xrayandclinical\SHAP_force.html
Mean AUC : 0.870467032967033
Std  AUC : 0.05928848948071798
